### **Parte 1 - Threads**
Threads são linhas de execução. Cada processo tem ao menos uma thread, mas pode conter várias, dividindo assim sua execução em várias tarefas menores.

Para criar uma thread, usamos a função *pthread_create*. Tal função retorna um int que indica o status da criação, sendo 0, se ela foi bem sucedida, ou outro valor, se houve algum problema. Além disso, ela tem os seguintes argumentos:
1. Ponteiro que armazena o identificador da thread;
2. Atributos da thread. Caso queira os atributos padrão, use NULL;
3. Ponteiro da função a ser usada para a thread. Tal função tem um único argumento: um ponteiro para void;
4. Um ponteiro para o argumento da função do ponto 3.

Outras funções úteis no gerenciamento das threads são a *pthread_exit*, a qual encerra a thread, e a *pthread_join*, que faz uma thread aguardar o encerramento de outra thread especificada. Veja a utilização dessas funções no exemplo abaixo, que faz *n* threads imprimirem uma mensagem na tela.





### **Parte 2 - Exercícios**


**Exercício 1** (0,75)

Comente a linha *pthread_join(threads[i],NULL)* no código *helloThreads.c* e veja o que acontece. Faça modificações no código para obter novamente a saída original sem necessitar usar um *pthread_join*. Dica: basta adicionar uma linha ao código.



---




In [ ]:
%%writefile helloThreads.c
#include <pthread.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

// Variavel global que define o numero de threads
int numberOfThreads=5;

// Funcao que as threads vao usar para dar um ola
void *printHello(void *tid){
  printf("Ola, eu sou a thread %d.\n", (int)(size_t) tid);
  pthread_exit(NULL);
}

int main(int argc, char **argv)
{
  pthread_t threads[numberOfThreads];
  for (int i=0;i<numberOfThreads;i++){
    // Criando uma thread e passando para ela a funcao printHello
    int status=pthread_create(&threads[i],NULL,printHello,(void *)(size_t) i);

    // Se o status nao for 0, houve um erro na criacao da thread
    if(status!=0){
      printf("Erro na criacao da thread %d. Codigo de erro %d.",i, status);
      return 1;
    }

    // Faz a thread principal esperar pela thread filha. Similar a funcao wait.
    //pthread_join(threads[i],NULL);
  }

  sleep(1); 

  return 0;
}

Overwriting helloThreads.c


In [6]:
!gcc ./helloThreads.c -o helloThreads
!./helloThreads

Ola, eu sou a thread 0.
Ola, eu sou a thread 1.
Ola, eu sou a thread 3.
Ola, eu sou a thread 2.
Ola, eu sou a thread 4.


**Exercício 2** (0,75)

Refaça o exercício 2 do laboratório 2 usando threads.

In [7]:
%%writefile exercicio2.c
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

int n = 20; // quantidade de termos da sequência de Fibonacci
int fib[20]; // O array para armazenar a sequência de Fibonacci

// Funcao que a thread vai usar para gerar a sequencia
void *generateFibonacci(void *param) {
    fib[0] = 0;
    if (n > 1) {
        fib[1] = 1;
        for (int i = 2; i < n; i++) {
            fib[i] = fib[i-1] + fib[i-2];
        }
    }
    pthread_exit(0);
}

int main(int argc, char **argv) {
    pthread_t tid;
    pthread_attr_t attr;

    // Obtém os atributos padrão da thread
    pthread_attr_init(&attr);

    // Cria a thread e passa a função
    pthread_create(&tid, &attr, generateFibonacci, NULL);

    // Espera a thread terminar
    pthread_join(tid, NULL);

    // Imprime a sequência
    printf("Sequência de Fibonacci até n=%d (gerada pela thread):\n", n);
    for (int i = 0; i < n; i++) {
        printf("%d ", fib[i]);
    }
    printf("\n");

    return 0;
}

Writing exercicio2.c


In [8]:
!gcc ./exercicio2.c -o exercicio2
!./exercicio2

Sequência de Fibonacci até n=20 (gerada pela thread):
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 


**Exercício 3** (1,00)

Refaça o exercício 3 do laboratório 2 usando threads.


---

In [9]:
%%writefile exercicio3.c
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

// A variável global para a soma
long long sum = 0;
// O mutex para garantir a exclusão mútua na seção crítica
pthread_mutex_t lock;

// Função que cada thread vai executar para somar um número
void *add_to_sum(void *arg) {
    int i = (int)(size_t)arg;
    
    // Bloqueia o mutex para acessar a variável compartilhada
    pthread_mutex_lock(&lock);
    
    // Adiciona o número à soma global
    sum += i;
    
    // Desbloqueia o mutex
    pthread_mutex_unlock(&lock);
    
    return NULL;
}

int main(int argc, char **argv) {
    int n = 10;
    
    // Inicializa o mutex
    if (pthread_mutex_init(&lock, NULL) != 0) {
        printf("\n mutex init falhou\n");
        return 1;
    }

    pthread_t threads[n];

    // Loop para criar as threads
    for (int i = 1; i <= n; i++) {
        pthread_create(&threads[i-1], NULL, add_to_sum, (void *)(size_t)i);
    }

    // Espera todas as threads terminarem
    for (int i = 0; i < n; i++) {
        pthread_join(threads[i], NULL);
    }

    // Imprime o resultado final
    printf("Soma de 1 até %d = %lld\n", n, sum);

    // Destrói o mutex
    pthread_mutex_destroy(&lock);

    return 0;
}

Writing exercicio3.c


In [10]:
!gcc ./exercicio3.c -o exercicio3
!./exercicio3

Soma de 1 até 10 = 55
